In [ ]:
import inspect

In [ ]:
from NPC import enemy

In [ ]:
print(inspect.signature(enemy));
#print(inspect.getfullargspec(enemy));

## Initialization

In [ ]:
filename = "Sample_Encounter_1"

### Reading enemies

In [ ]:
# List of enemies
enemies = []

with open(filename) as file:
    x = file.readline().split();
    try:
        nb_enemies = int(x[0])
    except ValueError:
        print("First line should be number of enemies followed by legend.")
    
    for i in range(nb_enemies):
        # Read enemy (1 line per enemy)
        NAME, AT, PRD, EV, PR, WEAPON, DMG_DICE_N, DMG_BONUS, COU, EXP = file.readline().split();
        # Convert specific variables to int for further use
        AT, PRD, EV, PR, DMG_DICE_N, DMG_BONUS, COU, EXP = \
            int(AT), int(PRD), int(EV), int(PR), int(DMG_DICE_N), int(DMG_BONUS), int(COU), int(EXP)
        # Create enemy instance
        myVars = vars()
        myVars[NAME] = enemy(NAME, AT, PRD, EV, PR, WEAPON, DMG_DICE_N, DMG_BONUS, COU, EXP)
        enemies.append(myVars[NAME])

In [ ]:
for opponent in enemies:
    print(opponent.info(),"\n")

## Fight Sequence

In [ ]:
# Temporary, for testing in absence of real players
import random

In [ ]:
# Attack
for opponent in enemies:
    opponent.attack();
    print();
    
# If counter-attacked
print("--------------------");
print("If counter-attacked:\n");
for opponent in enemies:
    counter = random.randint(1,10);
    opponent.countered(counter);
    print();

print([bad_guy.NAME for bad_guy in enemies]); # Names For Counter (if needed)

In [ ]:
# for taking counter hits - depends on player throw
hit=5
opponent = Rat

opponent.take_damage(hit-opponent.PR)
opponent.next_phase();

In [ ]:
# Defend when attacked
for opponent in enemies:
    hit = random.randint(1,10);
    opponent.defend(hit);
    print();